<a href="https://colab.research.google.com/github/Haz-mor/Haz-projects/blob/main/Sprint_10_Parte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🎯 Objetivo del ejercicio:
Aplicar los contenidos vistos sobre estandarización, transformación y métricas.

## 📝 Descripción del problema:
Dentro de la empresa Alfa Bank necesitamos predecir si un cliente dejará el banco pronto.Tenemos los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco.

*   RowNumber: índice de cadena de datos
*   CustomerId: identificador de cliente único
*   Surname: apellido
*   CreditScore: valor de crédito
*   Geography: país de residencia
*   Gender: sexo
*   Age: edad
*   Tenure: período durante el cual ha madurado el depósito a plazo fijo de un cliente (años)
*   Balance: saldo de la cuenta
*   NumOfProducts: número de productos bancarios utilizados por el cliente
*   HasCrCard: el cliente tiene una tarjeta de crédito (1 - sí; 0 - no)
*   IsActiveMember: actividad del cliente (1 - sí; 0 - no)
*   EstimatedSalary: salario estimado

In [ ]:
import pandas as pd

# Cargar el CSV
df = pd.read_csv("Churn.csv")

# Mostrar las primeras filas y columnas disponibles
df.dropna(inplace=True)
df.head()


## Antes de empezar!!... Hay columnas que podemos eliminar ? que no aporten valor al modelo ?

In [ ]:
### R:
col_to_del = []
df.drop(columns=col_to_del,inplace = True)

## Codificación de Variables Categóricas

### One-Hot Encoding: Para Variables Nominales!

#### Usando pandas : get_dummies()

In [ ]:
#Seleccionar columnas nominales :
col_to_transform = []
# Aplicamos onehotencoding de pandas:
df_transf = pd.get_dummies(df[col_to_transform], dtype=int, drop_first=True)

In [ ]:
# Mostramos la información
df_transf

In [ ]:
# Juntamos los dataframes
df_final = pd.concat([df,df_transf],axis=1)

In [ ]:
# Eliminamos las columnas transformadas
df_final.drop(columns=col_to_transform,inplace=True)

In [ ]:
# Mostramos el resultado:
df_final.head()

### Hay Variables Ordinales?

In [ ]:
## Respuesta:

## Estandarización de Variables Numéricas


### Escalado Estandar: StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

# Escalado de todas las variables (después de codificar) sin incluir la variable a predecir!
scaler_std = StandardScaler()

# Capturamos todas las columnas:
cols = list(df_final.columns)
cols.remove('Exited')
X_scaled = scaler_std.fit_transform(df_final[cols])
# Para inspeccionar el resultado
X_scaled_std_df = pd.DataFrame(X_scaled, columns=cols)

In [ ]:
X_scaled_std_df

### Minmax Scaler:

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_mm = MinMaxScaler()
X_scaled_mm = scaler_mm.fit_transform(df_final[cols])

# Para inspeccionar el resultado
X_scaled_mm_df = pd.DataFrame(X_scaled_mm, columns=cols)

In [ ]:
X_scaled_mm_df

## División de datos y Entrenamiento Básico

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

y = df['Exited']
X_train, X_test, y_train, y_test = train_test_split(X_scaled_std_df, y, test_size=0.3, random_state=42, stratify=y)


## Métricas de Clasificación

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
## Modelo simple:
model = LogisticRegression()
model.fit(X_train, y_train)

#Entrenamos:
y_pred = model.predict(X_test)

print("Reporte de métricas:")
print(classification_report(y_test, y_pred))

ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

## Curva ROC y AUC

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

y_proba = model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_proba)

plt.plot(fpr, tpr, label=f'AUC = {roc_auc_score(y_test, y_proba):.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend()
plt.grid(True)
plt.show()


## Manejo de Desbalanceo con class_weight

In [ ]:
# Modelo simple balanceado:
model_balanced = LogisticRegression(class_weight='balanced')
#Entrenamos:
model_balanced.fit(X_train, y_train)
print("Reporte con pesos balanceados:")
y_pred_new = model_balanced.predict(X_test)
print(classification_report(y_test, model_balanced.predict(X_test)))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_new)

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

y_proba = model_balanced.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_proba)

plt.plot(fpr, tpr, label=f'AUC = {roc_auc_score(y_test, y_proba):.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend()
plt.grid(True)
plt.show()